In [17]:
import os
import cv2
import shutil
import requests
from configparser import ConfigParser
from sklearn.model_selection import train_test_split as sk

# Config

In [18]:
parser = ConfigParser()
parser.read('dev.ini')
#print(parser.sections())
#parser.options('settings')
end_folder_name = parser.get('settings', 'dest_name')
create_folder = parser.getboolean('settings', 'create_folder', fallback = True)
path= parser.get('settings', 'path', fallback = os.getcwd()) #+ '\\' + end_folder_name
th = parser.getint('settings', 'size_treshold', fallback = 9999)
timeout = parser.getint('settings', 'timeout', fallback = 20)
clean_src = parser.getboolean('settings', 'clean_src', fallback = False)
neuron_number = parser.getint('settings', 'neuron_number', fallback = 0)
train_split = parser.getfloat('settings', 'train_split' , fallback = 0.7)
val_split = parser.getfloat('settings', 'val_split' , fallback = 0.1)
test_split = parser.getfloat('settings', 'test_split' , fallback = 0.2)

In [19]:
path

'C:\\Users\\Julian\\Documents\\GitHub\\ppgn'

# Get images and cut+resize

In [20]:
if create_folder:
    try:
        os.mkdir(os.path.join(path,end_folder_name))
        dest_folder_path = os.path.join(path,end_folder_name)
    except:
        dest_folder_path = os.path.join(path,end_folder_name)
        
try:
    source = os.mkdir(os.path.join(path,'Source'))
except Exception as e:
    source = os.path.join(path,'Source')
    print(e)

[WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'C:\\Users\\Julian\\Documents\\GitHub\\ppgn\\Source'


In [21]:
dest_folder_path

'C:\\Users\\Julian\\Documents\\GitHub\\ppgn\\test_cat__'

In [22]:
#Cutting image function
def cutting(img):
    
    width, height = img.shape[:2]
    
    if width >= th or height>= th:
        pass
    else:
        if width > height:
            x = int((width - height)/2)
            img = img[x:x+height, 0:height]
        else:
            x = int((height - width)/2)
            img = img[0:width, x:x + width] 
    
    return img   

In [23]:
content = []
with open(os.path.join(path,'url.txt'),encoding="utf8") as ff:
    content = ff.readlines()

In [24]:
#Delete duplicates
content = list(set(content))

In [25]:
'''
if clean_src:
    for item in os.listdir(source):
        os.remove(os.path.join(source, item))
'''

'\nif clean_src:\n    for item in os.listdir(source):\n        os.remove(os.path.join(source, item))\n'

In [26]:
i = 0

for num, pic in enumerate(os.listdir(source)): 
    
    '''
        
    try:
        img_data = requests.get(url, timeout = timeout).content
        
    except requests.exceptions.RequestException as e:
        print(e)
        continue
        
    '''
        
    try:
        
        i = str(num).zfill(5)
        name = os.path.join(source, pic )
    
        '''
        with open(name, 'wb') as handler:
            handler.write(img_data)
            
        '''
  

        print(name)

        img = cv2.imread(name)
        
    
        img = cutting(img)

        
        res = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_AREA)

        cv2.imwrite(os.path.join(dest_folder_path, str(i) + '.JPEG'),res)
        
 
    except Exception as e:
        print(e)

C:\Users\Julian\Documents\GitHub\ppgn\Source\20171113_150950.jpg
C:\Users\Julian\Documents\GitHub\ppgn\Source\20171116_231652.jpg
C:\Users\Julian\Documents\GitHub\ppgn\Source\20171117_150059.jpg
C:\Users\Julian\Documents\GitHub\ppgn\Source\20171129_174426.jpg
C:\Users\Julian\Documents\GitHub\ppgn\Source\20171130_115055.jpg
C:\Users\Julian\Documents\GitHub\ppgn\Source\20171130_130809.jpg


In [27]:
ff.close()

# Train/Val/Test Split

In [28]:
all_pics = os.listdir(dest_folder_path)
all_pics = [file for file in all_pics if file.endswith('.JPEG')]

['00000.JPEG', '00001.JPEG', '00002.JPEG', '00003.JPEG', '00004.JPEG', '00005.JPEG']


In [29]:
y = [neuron_number] * len(all_pics)
train, test, y_train, y_test = sk(all_pics, y, test_size=test_split, random_state=1)
train, val, y_val, y_val = sk(train, y_train, test_size=val_split, random_state=1)

In [30]:
#train folder
train.sort()

try:
    os.mkdir(os.path.join(dest_folder_path,'train'))
    pass
except Exception as e:
    print(e)
    
train_path = os.path.join(dest_folder_path,'train')


f= open(os.path.join(train_path,"_img_names.txt"),"w+")

    
for pic in train:
    try:
            
        shutil.move(os.path.join(dest_folder_path,pic), train_path)
        
        f.write(end_folder_name + '_' + str(pic) + ' ' + str(neuron_number) + ' \n')
        
    except Exception as e:
        
        print(e)

f.close()

In [31]:
#validation folder
val.sort()

try:
    os.mkdir(os.path.join(dest_folder_path,'validation'))
    pass
except Exception as e:
    print(e)
    
val_path = os.path.join(dest_folder_path,'validation')


f= open(os.path.join(val_path,"_img_names.txt"),"w+")
    
    
for pic in val:
    try:
        
        shutil.move(os.path.join(dest_folder_path,pic), val_path)
        
        f.write(end_folder_name + '_' + str(pic) + ' ' + str(neuron_number) + ' \n')
        
    except Exception as e:
        
        print(e)
        
f.close()

In [32]:
#train folder
test.sort()

try:
    os.mkdir(os.path.join(dest_folder_path,'test'))
    pass
except Exception as e:
    print(e)
    
    
test_path = os.path.join(dest_folder_path,'test')


f= open(os.path.join(test_path,"_img_names.txt"),"w+")
    
    
for pic in test:
    try:
        
        shutil.move(os.path.join(dest_folder_path,pic), test_path)
        
        f.write(end_folder_name + '_' + str(pic) + ' ' + str(neuron_number) + ' \n')
         
    except Exception as e:
        
        print(e)
        
f.close()